In [3]:
from langchain_community.llms import Ollama
from langchain import PromptTemplate

llm = Ollama(model="llama3.1:8b", stop=["<|eot_id|>"])

In [79]:
def get_model_response(user_prompt,system_prompt):
    template = """
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    {system_prompt}
    <|eot_id|>
    <|start_header_id|>user<|end_header_id|>
    {user_prompt}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """

    prompt = PromptTemplate(
    input_variables=["system_prompt","user_prompt"],
    template = template
    )

    response = llm(prompt.format(system_prompt = system_prompt, user_prompt=user_prompt))

    return response

In [130]:
scene_graph = {"countertop": ["bowl", "stove burner", "coffee machine", "pot", "pan","dish sponge"], "cabinet": ["microwave"], "table": ["house plant"], "floor": ["chair", "fridge"], "countertop_1": ["potato", "soap bottle", "lettuce"], "stove burner": ["salt shaker"], "countertop_3": ["plate"], "table_1": ["bread", "statue"]}

user_input = "모든 요리도구들을 같은 곳에 모아줘."

In [133]:
def generate_task_plans_prompt(scene_graph, user_input):
    template = f"""
    Mission : 
    You're tasked with generating plans to follow user's instruction.

    Input : 
    1. You'll receive a user's instruction which is a task you have to conduct.
    2. You'll receive a scene graph represented as a Python dictionary. Each key represents a workspace, and the corresponding value is a list of objects currently associated with that workspace. 

    Rules :
    1. You should make excellent plans to conduct user's instruction.
    2. You must only include objects and workspaces in the given scene graph in your plans.
    3. The plans can include moving objects to proper workspace.
    4. If the target location in the user's instruction is not in the input scene graph, aim for the location in the scene graph that is most similar to the target location.
    5. When moving objects, refrain from moving objects that are too heavy or large.

    Output :
    1. You'll output a plan to conduct user's instruction in form of sentences.
    2. You have to make a sufficient number of plans to follow user's instruction.

    ---------------------------------------------------------------------------------
    Example :
    [Input] : 
    [USER INSTRUCTION]
    Move the study materials on the desk and move all the objects away from couch.

    [Scene Graph]
    {{'counter' : ['book','pencil'],
    'couch' : ['remote controller','apple'],
    'couch_1' : ['toy']
    'table' : ['potted plant','bowl','spoon'],
    'desk' : ['laptop','pillow']}}
    
    [Process]
    Information analysis :
    1. Workspace identification : I can see workspaces 'counter', 'couch', 'table', 'desk' in the scene graph.
    2. Objects identification :
    (1) 'book', 'pencil' in workspace 'counter'
    (2) 'remote controller', 'apple' in workspace 'couch'
    (3) 'toy' in workspace' 'couch_1'
    (3) 'potted plant', 'bowl', 'spoon' in workspace 'table'
    (4) 'laptop', 'pillow' in workspace 'desk'

    Reasoning :
    There are two missions to solve. First is to move the study materials on the desk, and second is to move all the objects away from couch.
    I'll divide these two missions and make plans to solve it.
    
    First mission : move the study materials on the desk
    This is how i think to solve the mission.
    'book', 'pencil', 'laptop' are study materials in the scene. -> 'book' and 'pencil' are in 'counter' currently. -> I should move them to 'desk'.
    -> 'laptop' is a study material too. -> It is currently on 'desk' so it is in the right place. -> It doesn't have to me moved.

    Second mission : move all the objects away from couch.
    This is how i think to solve the mission.
    There are 'remote controller' and 'apple' in couch. -> They should be moved to other workspaces -> 'remote controller' should be moved to 'table' since someone would use it in the future.
    -> 'apple' should be moved to 'counter' since it's a food ingredient. -> 'toy' is in couch_1 and 'toy' should also be moved to other workspaces -> 'toy' should be moved to 'table' that someone would use it in the future. 

    [Output] :
    Step1. Move 'book' to the desk since it is a study material.
    Step2. Move 'pencil' to the desk since it is a study material.
    Step3. Move 'remote controller' to 'table' away from 'couch'
    Step4. Move 'apple' to 'counter' away from 'couch'
    Step5. Move 'toy' to 'table' from 'couch_1'
    
    ---------------------------------------------------------------------------------
    
    [User] 
    [Input] : 
    [USER INSTRUCTION]
    {user_input}

    [Scene Graph]  
    {scene_graph}

    [Process]

    [Output] : * If the target location in the user's instruction is not in the input scene graph, aim for the location in the scene graph that is the most similar to the target location.
    Step1.
    Step2.
    Step3.
    ...
    """
    return template

In [135]:
user_prompt = generate_task_plans_prompt(scene_graph, user_input)
response_1 = get_model_response(user_prompt,"주어진 한국어 명령과 scene graph를 이용해서, task planning을 해줘.")
print(response_1)

I'll do my best to provide a plan to follow your instruction based on the given user's instruction and scene graph.

**Input**

User Instruction: 모든 요리도구들을 같은 곳에 모아줘. (Korean for "Put all kitchen utensils in one place.")

Scene Graph:

```python
{
    'countertop': ['bowl', 'stove burner', 'coffee machine', 'pot', 'pan', 'dish sponge'],
    'cabinet': ['microwave'],
    'table': ['house plant'],
    'floor': ['chair', 'fridge'],
    'countertop_1': ['potato', 'soap bottle', 'lettuce'],
    'stove burner': ['salt shaker'],
    'countertop_3': ['plate'],
    'table_1': ['bread', 'statue']
}
```

**Process**

Information analysis:

* Workspace identification: I can see workspaces `countertop`, `cabinet`, `table`, `floor`, `countertop_1`, `stove burner`, `countertop_3`, and `table_1` in the scene graph.
* Objects identification:
	+ 'bowl', 'pot', 'pan', 'dish sponge' are kitchen utensils on `countertop`.
	+ 'microwave' is a kitchen appliance on `cabinet`.
	+ 'stove burner' has a 'salt shak

In [136]:
def preprocess_task_plans_prompt(task_plan):
    template = f"""
    Trim the given text below and write the step by step task plan in the given format as output. 
    Output should contain only the step-by-step sequence such as Step1, Step2, Step3, etc., without any other modifiers.

    
    [Text]
    {task_plan}
    
    [Format]
    Step1.
    Step2.
    Step3.
    ...
    """
    return template

In [137]:
user_prompt = preprocess_task_plans_prompt(response_1)
response_2 = get_model_response(user_prompt,"Please trim the given text following the user prompt")
print(response_2)

Step1. Move 'bowl', 'pot', 'pan', and 'dish sponge' to the `countertop`.

Step2. Move 'salt shaker' to the `countertop`.

Step3. Move 'microwave' to the `countertop`.


In [138]:
def generate_steps_code_prompt(plans): # 0802
    template = f"""
    Mission : 
    You're tasked with generating sequence of steps in form of python dictionary form with the action functions you can use to conduct a task given by user.

    Input : 
    1. You'll receive a step-by-step task plan to conduct the task.

    Action function :
    1. GoTo(object) : 'object' parameter can be either the name of 'place' or 'object'. The robot agent will go in front of the 'place' or 'object' utilizing this function.
    2. Pickup_Object(object) : 'object' parameter is the name of 'object'. You'll pick up the object utilizing this function.
    3. Put_Object(object) : 'object' parameter is the name of 'object'. You should go to 'place' or 'object' first, then you'll put the holding object at the 'place' or near the 'object'.

    Output :
    1. You'll generate a python dictionary where each key of the dictionary represents each step, and each value represents a list of action functions to perform each step.
    2. Make sure that you only include the Action functions mentioned.
    3. Output should contain only the final dictionary, without any other modifiers.

    
    Example :
    [Input] : 
    Step1. Move 'book' to the desk since it is a study material.
    Step2. Move 'pencil' to the desk since it is a study material.
    Step3. Move 'remote controller' to 'table' away from 'couch'
    Step4. Move 'apple' to 'counter' away from 'couch'
    Step5. Move 'toy' to 'table' from 'couch_1'

    [Output] :
    {{"Step1": [GoTo(book), Pickup_Object(book), GoTo(desk), Put_Object(book)]
    "Step2" : [GoTo(pencil), Pickup_Object(pencil), GoTo(desk), Put_Object(pencil)], 
    "Step3": [GoTo(remote controller), Pickup_Object(remote controller),GoTo(table),Put_Object(remote controller)],
    "Step4": [GoTo(apple), Pickup_Object(apple),GoTo(counter),Put_Object(apple)],
    "Step5": [GoTo(toy), Pickup_Object(toy),GoTo(table),Put_Object(toy)]}}

    [User] 
    [Input] : 
    {plans}

    [Output] :
    """
    return template

In [139]:
user_prompt = generate_steps_code_prompt(response_2)
print(user_prompt)


    Mission : 
    You're tasked with generating sequence of steps in form of python dictionary form with the action functions you can use to conduct a task given by user.

    Input : 
    1. You'll receive a step-by-step task plan to conduct the task.

    Action function :
    1. GoTo(object) : 'object' parameter can be either the name of 'place' or 'object'. The robot agent will go in front of the 'place' or 'object' utilizing this function.
    2. Pickup_Object(object) : 'object' parameter is the name of 'object'. You'll pick up the object utilizing this function.
    3. Put_Object(object) : 'object' parameter is the name of 'object'. You should go to 'place' or 'object' first, then you'll put the holding object at the 'place' or near the 'object'.

    Output :
    1. You'll generate a python dictionary where each key of the dictionary represents each step, and each value represents a list of action functions to perform each step.
    2. Make sure that you only include the Actio

In [140]:
response_3 = get_model_response(user_prompt,"Please preprocess the given task plan following the user prompt")
print(response_3)

Here is a Python solution for the given task:

```python
def process_task(task_plan):
    result = {}
    steps = task_plan.split('\n')
    for i, step in enumerate(steps):
        action = 'Move'
        target = ''
        place = ''
        objects = []
        
        # Extract information from the step description
        if 'to' in step:
            words = step.split(' to ')
            target = words[0].strip()
            place = words[1].strip()
        elif 'from' in step:
            words = step.split(' from ')
            target = words[0].strip()
            objects = words[1].split(', ')
        
        # Create action list
        result[f"Step{i+1}"] = []
        
        if len(objects) > 0:
            for obj in objects:
                result[f"Step{i+1}"].append(f"GoTo({obj})")
                result[f"Step{i+1}"].append(f"Pickup_Object({obj})")
                result[f"Step{i+1}"].append(f"GoTo(countertop)")
                result[f"Step{i+1}"].append(f"Put_Ob

In [144]:
def generate_final_codes_prompt(task_plan): # 0802
    template = f"""
    Trim the given text below and output the final dictionary as a string. 
    Output should contain only the dictionary with the given [Format] without any other modifiers.

    [Text]
    {task_plan}
    
    [Format]
    {{"Step1":[~~],"Step2":[~~]}}
    """
    return template

In [145]:
user_prompt = generate_final_codes_prompt(response_3)
print(user_prompt)


    Trim the given text below and output the final dictionary as a string. 
    Output should contain only the dictionary with the given [Format] without any other modifiers.

    [Text]
    Here is a Python solution for the given task:

```python
def process_task(task_plan):
    result = {}
    steps = task_plan.split('\n')
    for i, step in enumerate(steps):
        action = 'Move'
        target = ''
        place = ''
        objects = []
        
        # Extract information from the step description
        if 'to' in step:
            words = step.split(' to ')
            target = words[0].strip()
            place = words[1].strip()
        elif 'from' in step:
            words = step.split(' from ')
            target = words[0].strip()
            objects = words[1].split(', ')
        
        # Create action list
        result[f"Step{i+1}"] = []
        
        if len(objects) > 0:
            for obj in objects:
                result[f"Step{i+1}"].append(f"GoTo({ob

In [146]:
response_4 = get_model_response(user_prompt,"Please preprocess the given task plan following the user prompt")
print(response_4)

Here is the preprocessed text:


{"Step1":["GoTo(bowl)","Pickup_Object(bowl)","GoTo(countertop)","Put_Object(bowl)"],
"Step2":["GoTo(salt shaker)","Pickup_Object(salt shaker)","GoTo(countertop)","Put_Object(salt shaker)"],
"Step3":["GoTo(microwave)","Pickup_Object(microwave)","GoTo(countertop)","Put_Object(microwave)"]}


In [148]:
import ast

dict_start = response_4.find('{')
dict_end = response_4.rfind('}') + 1
dict_str = response_4[dict_start:dict_end]

# 문자열을 파이썬 딕셔너리로 변환
data = ast.literal_eval(dict_str)

# 결과 출력
print(data)

{'Step1': ['GoTo(bowl)', 'Pickup_Object(bowl)', 'GoTo(countertop)', 'Put_Object(bowl)'], 'Step2': ['GoTo(salt shaker)', 'Pickup_Object(salt shaker)', 'GoTo(countertop)', 'Put_Object(salt shaker)'], 'Step3': ['GoTo(microwave)', 'Pickup_Object(microwave)', 'GoTo(countertop)', 'Put_Object(microwave)']}
